In [1]:


// a few dependencies for the notebook:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
import $ivy.`com.lihaoyi::scalatags:0.12.0`

import better.files._
import better.files.Dsl._ 

// load historia code

val jarpath = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
assert(File(jarpath).exists, "must run jupyter notebook from docker")
interp.load.cp(os.Path(jarpath))

val rerun = false

import $ivy.$                                         

import $ivy.$                             

import $ivy.$                              


import better.files._

import better.files.Dsl._ 

// load historia code


jarpath: String = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
rerun: Boolean = false

In [2]:
// helper functions to run experiments

import almond.interpreter.api.DisplayData
import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._

import edu.colorado.plv.bounder.ir.IntConst
import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull,PureExpr,Equals}
import edu.colorado.plv.bounder.lifestate.LifeState
import edu.colorado.plv.bounder.lifestate.LifeState.{Not,And,LSConstraint,NS}
import edu.colorado.plv.bounder.ir.{CIEnter,CIExit,CBEnter,CBExit}

import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil,Action} // Historia utilities
import upickle.default.read
import upickle.default.write
import edu.colorado.plv.bounder.symbolicexecutor.state.{DisallowedCallin,MemoryLeak,NamedPureVar,TopVal, InitialQueryWithStackTrace, Reachable,IntVal}
import edu.colorado.plv.bounder.lifestate.SAsyncTask
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.{AllMatchers,SJavaThreading,FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, 
                                           SDialog, SpecSignatures, SpecSpace, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSTrue, Signature,SubClassMatcher}
import edu.colorado.plv.bounder.symbolicexecutor.{PreciseApproxMode,LimitAppRecursionDropStatePolicy,LimitMsgCountDropStatePolicy,
                                                  DumpTraceAtLocationPolicy,LimitLocationVisitDropStatePolicy, LimitCallStringDropStatePolicy, 
                                                  LimitMaterializedFieldsDropStatePolicy}
import edu.colorado.plv.bounder.symbolicexecutor.Z3TimeoutBehavior

val v = NamedPureVar("v")
val s = NamedPureVar("s")
val m = NamedPureVar("m")
val p = NamedPureVar("p")
val a = NamedPureVar("a")
val f = NamedPureVar("f")
val l = NamedPureVar("l")
val t = NamedPureVar("t")
val r = NamedPureVar("r")
val d = NamedPureVar("d")

def getIsDocker():Boolean = {
    val res = BounderUtil.runCmdStdout("whoami")
    res.trim == "root"
}

val isDocker = getIsDocker()
// val isDocker = false //TODO: overridden

val historiaDir = if(isDocker) "/home/bounder" else "/Users/shawnmeier/Documents/source/historia/Historia/"

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR

def runHistoriaWithSpec(configPath:File, printThenDone:Boolean = false, outputMode:String = "MEM"):String = {
    val javaMemLimit=20 // Gb Note that this only limits JVM not JNI which can go significantly higher
    val historiaJar = jarpath
    val apkRootDir = "/Users/shawnmeier/Documents/data/historia_generalizability"
    val outDir = configPath.parent.toString
    val config = read[RunConfig](configPath.contentAsString)
    val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
    val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${configPath} -b ${apkRootDir} -u ${outDir} -o ${outputMode} --debug"
    
    if(printThenDone){
        println(cmd)
        cmd
    }else{
        //BounderUtil.runCmdStdout(cmd, Some("/Users/shawnmeier/software/z3/build"))
        BounderUtil.runCmdFileOut(cmd, configPath.parent).toString
    }
}

def runHistoriaInfo(configPath:File, allSpecs:Iterable[LSSpec], printThenDone:Boolean = false, outputMode:String = "MEM") = {
    val javaMemLimit=20 // Gb Note that this only limits JVM not JNI which can go significantly higher
    val historiaJar = jarpath
    val apkRootDir = "/Users/shawnmeier/Documents/data/historia_generalizability"
    val outDir = configPath.parent.toString
    val config = read[RunConfig](configPath.contentAsString)
    val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
    val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m info -c ${configPath} -b ${apkRootDir} -u ${outDir} -o ${outputMode}"
    
    if(printThenDone){
        println(cmd)
    }else{
        //BounderUtil.runCmdStdout(cmd, Some("/Users/shawnmeier/software/z3/build"))
        println(BounderUtil.runCmdFileOut(cmd, configPath.parent).toString)
    }
}
// def printOutput(

def runAndPrint(configPath:File, allSpecs:Iterable[LSSpec], printThenDone:Boolean = false, outputMode:String = "MEM"):String = {
    val res = runHistoriaWithSpec(configPath,printThenDone, outputMode)
    println("run result")
    println(res)   
    println("specified messages")
    val msgSigs = allSpecs.flatMap(spec =>
                Set(spec.target) ++ spec.pred.allMsg).map(msg => msg.identitySignature)
    println(msgSigs)
    println(msgSigs.size)
    res
}


val dataDir = if(isDocker) "/home/testApps/" else "/Users/shawnmeier/Documents/data/reach_24_data/"
val notebooksDir = if(isDocker) "/home/notebooks/" else s"${historiaDir}/notebooks/"


//implement in a class so any sequence gets converted to a table that can be displayed in jupyter
trait TableAble {
  def headers:List[String]   
  def values:List[Any]
}
case class Table(v:Seq[TableAble]){

}
Displayers.register(classOf[Table], (t: Table) => {
  import scalatags.Text.all._
  Map(
    "text/html" -> {
      table(cls:="table")(
        tr(t.v.head.headers.map(v => th(v))),
        for (row <- t.v) yield tr(
          row.values.map{
              case v:String => td(v)
              case v:Int => td(v)
              case v:Float => td(v)
          }
        )
      ).render
    }
  ).asJava
})

case class CategoryRow(categoryTitle:String, shortTitle:String, subMeasures: List[(String,Any)]) extends TableAble{
    override def headers:List[String] = subMeasures.map(_._1)
    override def values:List[Any] = subMeasures.map(_._2)
}

case class RuntimeRow(dir:File, catRows:List[CategoryRow]) extends TableAble{
    def addCategories(other:RuntimeRow):RuntimeRow = {
        assert(dir == other.dir, s"cannot combine non-matching directories: ${dir} and ${other.dir}")
        RuntimeRow(dir, catRows ++ other.catRows)
    }
    def benchmarkName:String = dir.toString.split("/").last
    override def headers:List[String] = "benchmark name" :: catRows.flatMap{row => row.headers.map{hdr => s"${row.shortTitle}:${hdr}"}}
    override def values:List[Any] = benchmarkName :: catRows.flatMap{row => row.values}
}

def avg(values:Seq[Int]):Float = {
    val sum = values.sum.toFloat
    sum / values.size.toFloat
}

def countRowMatches[T](in:Seq[(T,Seq[String])], matcher:scala.util.matching.Regex):Seq[(T,Int)] = {
    in.map{case (tfile, rows) => (tfile,rows.filter(matcher.matches).size)}
}

// read and print statistics about runtime traces
// should be a directory with logcat output one trace per file named "logcat.txt", "logcat1.txt" etc
// add "WITNESS" to the end of each logcat file where the issue was reached.
def runtimeStats(dir:File):RuntimeRow = {
    val traces = dir.glob("logcat*txt")
    val traceContents = traces.map{t => (t,t.contentAsString().split("\n").toList)}.toList
    val traceCount = traceContents.size
    println(s"found ${traceCount} runtime traces")

    // compute average callbacks
    val callbackCountPerTrace = avg(countRowMatches(traceContents, ".*histInstrumentation.*cb.*".r).map(_._2))

    // compute average callins
    val callinCountPerTrace = avg(countRowMatches(traceContents, ".*histInstrumentation.*ci.*".r).map(_._2))

    // compute new messages
    val newCountPerTrace = avg(countRowMatches(traceContents, ".*histInstrumentation.*new.*".r).map(_._2))

    val witnessCount = traceContents.filter{case (tfile, rows) => rows.exists{r => r.contains("WITNESS")}}.size
    RuntimeRow(dir, List(CategoryRow("Runtime Inst", "ri", List(
        ("Witness Ratio", s"${witnessCount}/${traceCount}"),
        ("cb(n)", callbackCountPerTrace),
        ("ci(n)", callinCountPerTrace),
        ("tot", callbackCountPerTrace + callinCountPerTrace + newCountPerTrace)
    ))))
}

// compareRun is file name of matching runtime trace
def wistoriaStats(expBase:File, outputDir:File, allSpecs:Set[LSSpec], compareRun:Option[String]):RuntimeRow = {
    val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
    if(result.witnessExplanation.isEmpty){
        throw new IllegalArgumentException("Witness explanation was empty")
    }
    val wit = result.witnessExplanation.head.futureTrace.map{_.toString}
    val msgSigs = allSpecs.flatMap(spec =>
            Set(spec.target) ++ spec.pred.allMsg).map(msg => msg.identitySignature)

    val rawCount = countRowMatches(Seq(("",wit)), "CBEnter.*".r)
    val cbCount = rawCount.head._2
    val ciCount = countRowMatches(Seq(("",wit)), "CIExit.*".r).head._2
    val newCount = countRowMatches(Seq(("",wit)), "TNew.*".r).head._2

    // print trace for visual comparison (TODO: perhaps automate at some point)
    println("\nwistoria output:")
    wit.foreach{println}
    if(compareRun.isDefined){
        val traceContents = (expBase / compareRun.get).contentAsString.split("\n")
        val onlyTraceRows = traceContents.filter{row => row.contains("histInstrumentation")}
        val onlyTraceRowsTrimmed = onlyTraceRows.map{row => row.split("histInstrumentation")(1)}
        println("\nrecorded history:")
        onlyTraceRowsTrimmed.foreach{println}
    }
    RuntimeRow(expBase, List(CategoryRow("Wistoria","wi",
        List(
            ("specified msgs(n)", msgSigs.size),
            ("realistic", if(compareRun.isDefined)"yes" else "no"),
            ("cb(n)", cbCount),
            ("ci(n)", ciCount),
            ("tot", cbCount + ciCount + newCount)
        )
    )))
}

def printSpec(s:LSSpec) = s match{
    case LSSpec(forall, exists, pred, tgt, _) => println(s"Ɐ${forall.mkString(",")} . ∃${exists.mkString(",")} . ${tgt} -[]-> ${pred} ")
}

Table(Seq(
    RuntimeRow(File("/foo/bar"), List(CategoryRow("c1","c1",List(("v1",1),("v2",2))), CategoryRow("c2","c2",List(("v3",3))))),
    RuntimeRow(File("/foo/baz"), List(CategoryRow("c1","c1",List(("v1",4),("v2",5))), CategoryRow("c2","c2",List(("v3",6))))),
))

benchmark name,c1:v1,c1:v2,c2:v3
bar,1,2,3
baz,4,5,6


import almond.interpreter.api.DisplayData

import jupyter.Displayer, jupyter.Displayers

import scala.collection.JavaConverters._


import edu.colorado.plv.bounder.ir.IntConst

import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull,PureExpr,Equals}

import edu.colorado.plv.bounder.lifestate.LifeState

import edu.colorado.plv.bounder.lifestate.LifeState.{Not,And,LSConstraint,NS}

import edu.colorado.plv.bounder.ir.{CIEnter,CIExit,CBEnter,CBExit}


import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil,Action} // Historia utilities

import upickle.default.read

import upickle.default.write

import edu.colorado.plv.bounder.symbolicexecutor.state.{DisallowedCallin,MemoryLeak,NamedPureVar,TopVal, InitialQueryWithStackTrace, Reachable,IntVal}

import edu.colorado.plv.bounder.lifestate.SAsyncTask

import edu.colorado.plv.bounder.PickleSpec

import edu.colorado.plv.bounder.lifestate.{AllMatchers,SJavaThreading,FragmentGetActivityNullSpec, LifeState, Lifecy

Memory Leak
-----------

In [23]:

// Location of the compilable application and trace files
// dataDir is "/home/testApps/" and may be opened in a terminal (+ next to this tab then select terminal)
// Application may be modified and re-run
val memLeakExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_activity_leak_noStatic"

//Load the runtime trace from the memory leak benchmark
val memLeakRuntimeStats = runtimeStats(File(memLeakExpBase))
Table(Seq(memLeakRuntimeStats))

found 2 runtime traces


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot
EXN_activity_leak_noStatic,1/2,5.5,2.0,11.5


memLeakExpBase: String = "/home/testApps//ChatGPT_Benchmarks/EXN_activity_leak_noStatic"
memLeakRuntimeStats: RuntimeRow = RuntimeRow(
  /home/testApps/ChatGPT_Benchmarks/EXN_activity_leak_noStatic,
  List(
    CategoryRow(
      "Runtime Inst",
      "ri",
      List(
        ("Witness Ratio", "1/2"),
        ("cb(n)", 5.5F),
        ("ci(n)", 2.0F),
        ("tot", 11.5F)
      )
    )
  )
)

In [25]:
// Memory Leak
object MemLeak{
    // Directory to store files generated by Wistoria
    val memLeakOutputDir = File(s"/home/notebooks/reachExpGPT/MemLeak")
    // The set of history implications used in this benchmark
    // LifecycleSpec is imported since we can reuse this from the Historia paper.
    // The output of each cell prints the specs when run.
    val memLeakAllSpecs = Set(LifecycleSpec.Activity_onDestroy_last)

    // messages may be defined and added as needed for the trace to be understandable.
    // CIExit defines that this is the exit of a callin (CBEnter and CBExit also exist for the entry and exit of callbacks)
    // SubClassMatcher says "match this method on each subclass of "Thread"
    // Methods are matched using a regular expression thus the escape chars on the parentheses
    // "Thread_start" is a unique identifier printed by the tool and may be more concise than the fully qualified name.
    // The last part "TopVal::t::Nil" defines the variables bound to each argument and return value (e.g. t represents the thread)
    // TopVal indicates that we don't care about that value and not to print it (e.g. to match void)
    val startMsg = AbsMsg(CIExit, SubClassMatcher("java.lang.Thread","void start\\(\\)", "Thread_start"), TopVal::t::Nil)
}

// Function used to hide Almond/Jupyter verboseness
def runMemLeak(outputDir:File, allSpecs:Set[LSSpec]){

    // Identify the target location and safety property
    val query = MemoryLeak("android.app.Activity", // type of object being leaked
        Signature("com.example.activityleak.LeakActivity$1", "void run()"),  // signature of leaky method 
        39, // line of leak
        SpecSignatures.Activity_onDestroy_exit, // Condition under which leak has occurred (O a.onDestroy() and some field pointing to a )
        NamedPureVar("a")) // target variable for the condition


    // location of the compiled apk
    val inputApk = s"${memLeakExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    // Write output such as the trace and full memory states to this location
    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/MemLeak")

    // A run configuration captures all the settings needed to run wistoria
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                    timeLimit=Int.MaxValue, // Give up after some amount of time
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), // Target location and safety property
                    truncateOut=false, // Turn on if disk I/O becomes limiting
                    specSet = PickleSpec(allSpecs, // Specs defining when the framework may take an action
                                             Set(), 
                                             Set(SJavaThreading.runnableI, MemLeak.startMsg) // Extra messages that we would like to display
                                        ),
                    componentFilter =  None, // Stop exploring if we go outside of a given package (e.g. Some(List("foo.bar.*")) means only consider code in the package "foo.bar", None means any code)
                    approxMode = PreciseApproxMode(true,  // Use the analysis in incorrectness logic mode
                                                   List(LimitLocationVisitDropStatePolicy(3), // Drop states if we visit a location 3 times
                                                              LimitAppRecursionDropStatePolicy(3),  // Drop methods if recursion occurs more than 3 times
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)))), // Drop states where any field is materialized more than twice
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy( // Z3 specific settings
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16)) // Run 16 instances of z3 simultaneously.
                        )


    if(!outputDir.isDirectory){  // make output directory if it doesn't exist
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json") 
    val configCfgPath = configOutputDir / "cfg.json"  
    cfgPath.overwrite(write(cfg)) // write run configuration to a file
    


    val justPrintCommand = false // set to true to see the command line invocation of Wistoria
    println(runAndPrint(cfgPath, allSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString)) // run wistoria

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runMemLeak(MemLeak.memLeakOutputDir, MemLeak.memLeakAllSpecs)
}


object MemLeakStats{  // print output and collect statistics about the run including the specs, output of Wistoria, and recorded trace
    println("specs:")
    MemLeak.memLeakAllSpecs.foreach{printSpec}
    val wistoria = wistoriaStats(File(memLeakExpBase),MemLeak.memLeakOutputDir, MemLeak.memLeakAllSpecs, Some("logcat.txt"))
}
val memLeakRow = Table(Seq(memLeakRuntimeStats.addCategories(MemLeakStats.wistoria)))

specs:
Ɐp-a . ∃ . O(CBEnter I_CBEnter_ActivityonDestroy ( _T_,p-a ) -[]-> NS( O(CBEnter I_CBEnter_ActivityonCreate ( _T_,p-a ) , O(CBExit I_CBExit_ActivityonDestroy ( _T_,p-a ) ) 

wistoria output:
CBEnter void onCreate(android.os.Bundle)( _T_,@2 )
TNew(@4,{1})
TNew(@3,{2})
CIExit void start()( _T_,@3 )
CBEnter void onDestroy()( _T_,@2 )
CBExit void onDestroy()( _T_,@2 )
CBEnter run()( _T_,@4 )

recorded history:
     com.example.activityleak             I  cb 146907790 onCreate
     com.example.activityleak             I   30255791 = new Runnable$1 146907790
     com.example.activityleak             I   15543996 = new Thread 146907790 
     com.example.activityleak             I  ci 15543996 start
     com.example.activityleak             I  cb 30255791 run
     com.example.activityleak             I  field this$0 146907790
     com.example.activityleak             I  cb 146907790onDestroy
     com.example.activityleak             I  cb 54828447 onCreate
     com.example.activityleak 

benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_activity_leak_noStatic,1/2,5.5,2.0,11.5,3,yes,3,1,6


defined object MemLeak
defined function runMemLeak
defined object MemLeakStats

Bitmap Mishandling
-----------------

In [5]:
// Bitmap mishandling runtime stats

val bitmapExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_bitmap_mishandling"
val bitmapRuntimeStats = runtimeStats(File(bitmapExpBase))
Table(Seq(bitmapRuntimeStats))

found 1 runtime traces


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot
EXN_bitmap_mishandling,1/1,15.0,32.0,54.0


bitmapExpBase: String = "/home/testApps//ChatGPT_Benchmarks/EXN_bitmap_mishandling"
bitmapRuntimeStats: RuntimeRow = RuntimeRow(
  /home/testApps/ChatGPT_Benchmarks/EXN_bitmap_mishandling,
  List(
    CategoryRow(
      "Runtime Inst",
      "ri",
      List(
        ("Witness Ratio", "1/1"),
        ("cb(n)", 15.0F),
        ("ci(n)", 32.0F),
        ("tot", 54.0F)
      )
    )
  )
)

In [6]:
object Bitmap{
    val setImageResource = (intval:Int) => AbsMsg(CIExit, 
                                                  SubClassMatcher("android.widget.ImageView","void setImageResource\\(int\\)","ImageView_setImageResource"), 
                                                  List[PureExpr](TopVal,TopVal,IntVal(intval)))

    val ArrayListType = "android.fake.IntArrayList"
    val ArrayListAddMsg = AbsMsg(CIExit, SubClassMatcher(ArrayListType, ".* add\\(.*", "ArrayList_add"), TopVal::a::v::Nil)
    val ArrayListGetMsg = AbsMsg(CIExit, SubClassMatcher(ArrayListType, ".* get\\(.*", "ArrayList_get"), v::a::TopVal::Nil)
    val getAfterAdd = LSSpec(a::v::Nil, Nil, ArrayListAddMsg, ArrayListGetMsg)

    val IntegerIntValue = AbsMsg(CIExit, SubClassMatcher("java.lang.Integer", "int intValue\\(.*", "Integer_intValue"), v::t::Nil )
    val intValEq = LSSpec(v::t::Nil, Nil, LSConstraint(v,Equals,t), IntegerIntValue)
    val BaseAdapterCallbacks = List("getCount","getItem","getItemId","getView")

    val BaseAdapter = "android.widget.BaseAdapter"
    val BaseAdapterMessages = BaseAdapterCallbacks.map{cbName => 
        (cbName,AbsMsg(CBEnter,SubClassMatcher(BaseAdapter, s".* ${cbName}\\(.*", "BaseAdapter_getView"), TopVal::a::Nil))}.toMap

    val BaseAdapter_getView_getCount = LSSpec(a::Nil, Nil, BaseAdapterMessages("getCount"), BaseAdapterMessages("getView"))

    val BaseAdapterInitCallin = AbsMsg(CIExit, SubClassMatcher(BaseAdapter, ".* <init>\\(.*", "BaseAdapter_init"), TopVal::a::Nil)

    val BaseAdapter_init_getCount = LSSpec(a::Nil, Nil, BaseAdapterInitCallin, BaseAdapterMessages("getCount"))

    val disallow = LSSpec(Nil,Nil, setImageResource(2130837508), setImageResource(2130837509).copy(mt = CIEnter))
    val outputDir = File(s"/home/notebooks/reachExpGPT/Bitmap")
    
    
    val enableSpecs = Set[LSSpec](intValEq, getAfterAdd, BaseAdapter_getView_getCount, BaseAdapter_init_getCount)
    
    val expBase = bitmapExpBase
}


def runBitmap(outputDir:File, enableSpecs:Set[LSSpec], disallow:LSSpec){

    
    val query = DisallowedCallin("com.example.bitmapmishandle.MainActivity$ImageAdapter", 
                                   "android.view.View getView(int,android.view.View,android.view.ViewGroup)", disallow)

    val inputApk = s"${Bitmap.expBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/Bitmap")
    val allMsg = Bitmap.BaseAdapterMessages.values.toSet + Bitmap.ArrayListAddMsg + Bitmap.ArrayListGetMsg + 
        Bitmap.IntegerIntValue + SpecSignatures.Activity_init_entry + Bitmap.BaseAdapterInitCallin + SpecSignatures.Activity_onCreate_entry

    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(enableSpecs, 
                                             Set(disallow), 
                                             allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs + disallow,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runBitmap(Bitmap.outputDir, Bitmap.enableSpecs, Bitmap.disallow) 
}


object BitmapStats{
    println("specs:")
    (Bitmap.enableSpecs + Bitmap.disallow).foreach{printSpec}
    val wistoria = wistoriaStats(File(bitmapExpBase),Bitmap.outputDir, Bitmap.enableSpecs + Bitmap.disallow, Some("logcat.txt"))
}
val bitmapRow = Table(Seq(bitmapRuntimeStats.addCategories(BitmapStats.wistoria)))

specs:
Ɐp-a,p-v . ∃ . O(CIExit I_CIExit_ArrayListget ( p-v,p-a,_T_ ) -[]-> O(CIExit I_CIExit_ArrayListadd ( _T_,p-a,p-v ) 
Ɐp-a . ∃ . O(CBEnter I_CBEnter_BaseAdaptergetView ( _T_,p-a ) -[]-> O(CIExit I_CIExit_BaseAdapterinit ( _T_,p-a ) 
Ɐ . ∃ . O(CIEnter I_CIEnter_ImageViewsetImageResource ( _T_,_T_,2130837509 ) -[]-> O(CIExit I_CIExit_ImageViewsetImageResource ( _T_,_T_,2130837508 ) 
Ɐp-v,p-t . ∃ . O(CIExit I_CIExit_IntegerintValue ( p-v,p-t ) -[]-> [ p-v == p-t ] 
Ɐp-a . ∃ . O(CBEnter I_CBEnter_BaseAdaptergetView ( _T_,p-a ) -[]-> O(CBEnter I_CBEnter_BaseAdaptergetView ( _T_,p-a ) 

wistoria output:
CBEnter void onCreate(android.os.Bundle)( _T_,@11 )
TNew(@12,{1})
CIExit  <init>(( _T_,@12 )
TNew(@0,{0})
CBEnter void <init>(( _T_,@0 )
TNew(@6,{2})
CIExit  add(( _T_,@6,@2 )
CIExit  add(( _T_,@6,@5 )
CIExit  add(( _T_,@6,@10 )
CIExit  add(( _T_,@6,@9 )
CIExit  add(( _T_,@6,@1 )
CBEnter  getCount(( _T_,@12 )
CBEnter  getView(( _T_,@12 )
CIExit  get(( @5,@6,_T_ )
CIExit void setImageReso

benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_bitmap_mishandling,1/1,15.0,32.0,54.0,7,yes,5,9,17


defined object Bitmap
defined function runBitmap
defined object BitmapStats

Dialog Origin
-------------

In [7]:
val dialogExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_dialog_originally_created_here"
val dialogRuntimeStats = runtimeStats(File(dialogExpBase))
Table(Seq(dialogRuntimeStats))

found 2 runtime traces


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot
EXN_dialog_originally_created_here,1/2,4.0,6.0,14.5


dialogExpBase: String = "/home/testApps//ChatGPT_Benchmarks/EXN_dialog_originally_created_here"
dialogRuntimeStats: RuntimeRow = RuntimeRow(
  /home/testApps/ChatGPT_Benchmarks/EXN_dialog_originally_created_here,
  List(
    CategoryRow(
      "Runtime Inst",
      "ri",
      List(
        ("Witness Ratio", "1/2"),
        ("cb(n)", 4.0F),
        ("ci(n)", 6.0F),
        ("tot", 14.5F)
      )
    )
  )
)

In [26]:
object DialogCreated{
    val dialogDismiss = AbsMsg(CIEnter, SDialog.dismissSignature, TopVal::d::Nil)
    val disallow = LSSpec(a::Nil, d::Nil, NS(SDialog.showI, dialogDismiss),SpecSignatures.Activity_onDestroy_exit)
    val enableSpecs = Set[LSSpec]()
    val outputDir = File("/home/notebooks/reachExpGPT/DialogCreated")
    val Thread_start = AbsMsg(CIExit, SubClassMatcher("java.lang.Thread", "void start\\(.*", "Thread_start"), TopVal::t::Nil)

    val DialogCallins2 = List("<init>","setMessage")
    val DialogCallins2Matcher = DialogCallins2.map{name => (name,
                                                            AbsMsg(CIExit, SubClassMatcher(SDialog.DialogC, s".* ${name}\\(.*", s"Dialog_${name}"), TopVal::d::a::Nil))}.toMap
    
    val allMsg = Set(SpecSignatures.Activity_onCreate_entry, SpecSignatures.Activity_onDestroy_entry, SJavaThreading.runnableI,Thread_start) ++ DialogCallins2Matcher.values
}
def runDialogCreated(){
    val outputDir = DialogCreated.outputDir
    val disallow = DialogCreated.disallow
    val enableSpecs = DialogCreated.enableSpecs

    
    val query = DisallowedCallin("com.example.bitmapmishandle.MainActivity", 
                                   "void onDestroy()", DialogCreated.disallow)

    // TODO: figure out why the android build changed this?
    // val inputApk = s"${Bitmap.expBase}/app_bug/app/build/intermediates/apk/debug/app-debug.apk"
    val inputApk = s"${dialogExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/DialogCreated")
    

    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(DialogCreated.enableSpecs, 
                                             Set(DialogCreated.disallow), 
                                             DialogCreated.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs + disallow,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runDialogCreated()
}

object DialogCreatedStats{
    println("specs:")
    (DialogCreated.enableSpecs + DialogCreated.disallow).foreach{printSpec}
    val wistoria = wistoriaStats(File(dialogExpBase),DialogCreated.outputDir, DialogCreated.enableSpecs + DialogCreated.disallow, Some("logcat2.txt"))
}
val dialogRow = Table(Seq(dialogRuntimeStats.addCategories(DialogCreatedStats.wistoria)))

specs:
Ɐp-a . ∃p-d . O(CBExit I_CBExit_ActivityonDestroy ( _T_,p-a ) -[]-> NS( O(CIExit I_CIExit_Dialogshow ( p-d,_T_,p-a ) , O(CIEnter I_CIEnter_Dialogdismiss ( _T_,p-d ) ) 

wistoria output:
CBEnter void onCreate(android.os.Bundle)( _T_,@7 )
TNew(@2,{1})
CIExit  <init>(( _T_,@2,@7 )
CIExit  setMessage(( _T_,@2,NULL )
CIExit show(( _T_,_T_ )
TNew(@4,{2})
CIExit void start(( _T_,@4 )
CBEnter void onDestroy()( _T_,@5 )

recorded history:
     com.example.bitmapmishandle          I  cb 88959629 onCreate 0
     com.example.bitmapmishandle          I  ci 88959629 setContentView 178413125
     com.example.bitmapmishandle          I   238639514 = new ProgressDialog 88959629
     com.example.bitmapmishandle          I  ci 238639514 setMessage248791755
     com.example.bitmapmishandle          I  ci 238639514 show 
     com.example.bitmapmishandle          I   88284195 = new MainActivity$1 88959629
     com.example.bitmapmishandle          I   104322336 = new Thread 88284195
     com.example.b

benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_dialog_originally_created_here,1/2,4.0,6.0,14.5,3,yes,2,4,8


defined object DialogCreated
defined function runDialogCreated
defined object DialogCreatedStats

## Execute Twice

In [9]:
val execTwiceBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_execute_twice"
val execTwiceRuntimeStats = runtimeStats(File(execTwiceBase))
Table(Seq(execTwiceRuntimeStats))

found 2 runtime traces


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot
EXN_execute_twice,1/2,9.5,15.5,26.0


execTwiceBase: String = "/home/testApps//ChatGPT_Benchmarks/EXN_execute_twice"
execTwiceRuntimeStats: RuntimeRow = RuntimeRow(
  /home/testApps/ChatGPT_Benchmarks/EXN_execute_twice,
  List(
    CategoryRow(
      "Runtime Inst",
      "ri",
      List(
        ("Witness Ratio", "1/2"),
        ("cb(n)", 9.5F),
        ("ci(n)", 15.5F),
        ("tot", 26.0F)
      )
    )
  )
)

In [27]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}

val task_get_status = AbsMsg(CIExit, SubClassMatcher(Set("android.os.AsyncTask"), "android.os.AsyncTask.Status getStatus\\(\\)", "taskGetStatus"), a::Nil)
val set_content_view = AbsMsg(CIExit, SubClassMatcher(Set("android.app.Activity"), "void setContentView\\(.*", "setContentView"), TopVal::v::Nil);
val find_view = AbsMsg(CIExit, SubClassMatcher(Set("android.app.Activity"), ".* findViewById\\(.*", "findViewById"),a::d::Nil)

object ExecTwice{
    val disallow = SAsyncTask.disallowDoubleExecute//LSSpec(a::Nil, d::Nil, SDialog.showI,SpecSignatures.Activity_onDestroy_exit)
    val enableSpecs = Set[LSSpec](ViewSpec.clickWhileNotDisabled)
    val outputDir = File("/home/notebooks/reachExpGPT/ExecTwice")
    // println(outputDir.exists())
   
    val allMsg = Set[LifeState.OAbsMsg](SpecSignatures.Activity_onCreate_entry, ViewSpec.onClickI, task_get_status,set_content_view, find_view)
}
def runExecTwice(){
    val outputDir = ExecTwice.outputDir
    val disallow = ExecTwice.disallow
    val enableSpecs = ExecTwice.enableSpecs

    
    val query = DisallowedCallin("com.example.bitmapmishandle.MainActivity$1", 
                                   "void onClick(android.view.View)", ExecTwice.disallow)

    val inputApk = s"${execTwiceBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/ExecTwice")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(ExecTwice.enableSpecs, 
                                             Set(ExecTwice.disallow), 
                                             ExecTwice.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs + disallow,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runExecTwice()
}

object ExecTwiceStats{
    println("specs:")
    (ExecTwice.enableSpecs + ExecTwice.disallow).foreach{printSpec}
    val wistoria = wistoriaStats(File(execTwiceBase),ExecTwice.outputDir, ExecTwice.enableSpecs + ExecTwice.disallow, Some("logcat.txt"))
}
val execRow =  Table(Seq(execTwiceRuntimeStats.addCategories(ExecTwiceStats.wistoria)))

specs:
Ɐp-l . ∃p-v . O(CBEnter I_CBEnter_ViewOnClickListeneronClick ( _T_,p-l ) -[]-> [ O(CIExit I_CIExit_ViewsetOnClickListener ( _T_,p-v,p-l ) && [(Not O(CIExit I_CIExit_ViewsetEnabled ( _T_,p-v,false )) || NS( O(CIExit I_CIExit_ViewsetEnabled ( _T_,p-v,true ) , O(CIExit I_CIExit_ViewsetEnabled ( _T_,p-v,false ) )] ] 
Ɐp-t . ∃ . O(CIEnter I_CIEnter_AsyncTaskexecute ( _T_,p-t ) -[]-> O(CIExit I_CIExit_AsyncTaskexecute ( _T_,p-t ) 

wistoria output:
CBEnter void onCreate(android.os.Bundle)( _T_,@7 )
TNew(@0,{4})
CIExit setOnClickListener(( _T_,@5,@0 )
CBEnter onClick(( _T_,@0,@2 )
CIExit AsyncTask execute()( _T_,@9 )
CBEnter onClick(( _T_,@0,@8 )
CIEnter AsyncTask execute()( _T_,@9 )

recorded history:
     com.example.bitmapmishandle          I  cb 16978004 onCreate
     com.example.bitmapmishandle          I  ci 16978004 setContentView 2131361820
     com.example.bitmapmishandle          I   166603391 = new MainActivity$DownloadTask 
     com.example.bitmapmishandle          I  14398

benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_execute_twice,1/2,9.5,15.5,26.0,5,yes,3,2,6


import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}


task_get_status: OAbsMsg = OAbsMsg(
  CIExit,
  SubClassMatcher(
    Set("android.os.AsyncTask"),
    "android.os.AsyncTask.Status getStatus\\(\\)",
    "taskGetStatus"
  ),
  List(NamedPureVar("a")),
  false
)
set_content_view: OAbsMsg = OAbsMsg(
  CIExit,
  SubClassMatcher(
    Set("android.app.Activity"),
    "void setContentView\\(.*",
    "setContentView"
  ),
  List(_T_, NamedPureVar("v")),
  false
)
find_view: OAbsMsg = OAbsMsg(
  CIExit,
  SubClassMatcher(
    Set("android.app.Activity"),
    ".* findViewById\\(.*",
    "findViewById"
  ),
  List(NamedPureVar("a"), NamedPureVar("d")),
  false
)
defined object ExecTwice
defined function runExecTwice
defined object ExecTwiceStats

## Fragment LifeCycle

In [11]:
val fragmentLFExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_fragment_lifecycle"
val fragmentLFRuntimeStats = runtimeStats(File(fragmentLFExpBase))
Table(Seq(fragmentLFRuntimeStats))

found 1 runtime traces


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot
EXN_fragment_lifecycle,1/1,6.0,7.0,14.0


fragmentLFExpBase: String = "/home/testApps//ChatGPT_Benchmarks/EXN_fragment_lifecycle"
fragmentLFRuntimeStats: RuntimeRow = RuntimeRow(
  /home/testApps/ChatGPT_Benchmarks/EXN_fragment_lifecycle,
  List(
    CategoryRow(
      "Runtime Inst",
      "ri",
      List(
        ("Witness Ratio", "1/1"),
        ("cb(n)", 6.0F),
        ("ci(n)", 7.0F),
        ("tot", 14.0F)
      )
    )
  )
)

In [28]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}

    val frag_destroy_view = AbsMsg(CBEnter, SubClassMatcher(SpecSignatures.Fragment,"void onDestroyView\\(\\)", "onDestroyView"), TopVal::a::Nil);
    val frag_create_view = AbsMsg(CBEnter, SubClassMatcher(SpecSignatures.Fragment,"android.view.View onCreateView\\(.*\\)", "onCreateView"), TopVal::a::d::Nil);

    val inflate = AbsMsg(CIExit, SubClassMatcher(Set("android.view.LayoutInflater"), "android.view.View inflate\\(.*\\)", "inflate"), v::Nil)
    val frag_des_last =  LSSpec(a::Nil,d::Nil,frag_create_view, frag_destroy_view)

    val frag_trans = Set("androidx.fragment.app.FragmentTransaction")
    val commit_transaction = AbsMsg(CIExit,SubClassMatcher(frag_trans,"int commit\\(\\)", "commit"),TopVal::f::Nil)
    val begin_transaction = AbsMsg(CIExit,SubClassMatcher(Set("androidx.fragment.app.FragmentManager"),"androidx.fragment.app.FragmentTransaction beginTransaction\\(\\)", "beginTrans"),f::TopVal::Nil)
    val replace = AbsMsg(CIExit, SubClassMatcher(frag_trans, "androidx.fragment.app.FragmentTransaction replace\\(.*\\)", "replace"), f::TopVal::TopVal::a::Nil)

    //Need to see the commit before onCreateView
    val commit_first = LSSpec(a::d::Nil, f::Nil, And(replace,commit_transaction), frag_create_view)
object FragmentLF{

    
    val disallow = LSSpec(a::Nil, d::Nil, SDialog.showI,SpecSignatures.Activity_onDestroy_exit)
    val enableSpecs = Set[LSSpec](frag_des_last, commit_first)
    val outputDir = File("/home/notebooks/reachExpGPT/FragmentLF")
    // println(outputDir.exists())
   
    val allMsg = Set[LifeState.OAbsMsg](frag_destroy_view, 
                                        frag_create_view,
                                        inflate,
                                        commit_transaction,
                                        replace,
                                        set_content_view,
                                        begin_transaction,
                                        SpecSignatures.Activity_onCreate_entry)
}
def runFragmentLF(){
    val outputDir = FragmentLF.outputDir
    val disallow = FragmentLF.disallow
    val enableSpecs = FragmentLF.enableSpecs

    
    val query = MemoryLeak("android.view.View", LifeState.Signature("com.example.fragmentlifecycle.ArticlesFragment", "void onDestroyView()"), 42,
                           
                           And(And(inflate,frag_destroy_view), frag_create_view),v)
    //Reachable(LifeState.Signature("com.example.fragmentlifecycle.ArticlesFragment", "void onDestroyView()"),42);

    val inputApk = s"${fragmentLFExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/FragmentLF")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(FragmentLF.enableSpecs, 
                                             Set(), 
                                             FragmentLF.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=12))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    
    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runFragmentLF()
}

object FragmentLFStats{
    println("specs:")
    (FragmentLF.enableSpecs).foreach{printSpec}
    val wistoria = wistoriaStats(File(fragmentLFExpBase),FragmentLF.outputDir, FragmentLF.enableSpecs, Some("logcat.txt"))
}
val fragRow = Table(Seq(fragmentLFRuntimeStats.addCategories(FragmentLFStats.wistoria)))

specs:
Ɐp-a . ∃p-d . O(CBEnter I_CBEnter_onDestroyView ( _T_,p-a ) -[]-> O(CBEnter I_CBEnter_onCreateView ( _T_,p-a,p-d ) 
Ɐp-a,p-d . ∃p-f . O(CBEnter I_CBEnter_onCreateView ( _T_,p-a,p-d ) -[]-> [ O(CIExit I_CIExit_replace ( p-f,_T_,_T_,p-a ) && O(CIExit I_CIExit_commit ( _T_,p-f ) ] 

wistoria output:
CBEnter void onCreate(android.os.Bundle)( _T_,@2 )
CIExit void setContentView(( _T_,@2 )
CIExit androidx.fragment.app.FragmentTransaction beginTransaction()( @1,_T_ )
TNew(@5,{2})
CIExit androidx.fragment.app.FragmentTransaction replace()( @7,_T_,_T_,@5 )
CIExit int commit()( _T_,@7 )
CBEnter android.view.View onCreateView()( _T_,@5,@9 )
CIExit android.view.View inflate()( @4 )
CBEnter void onDestroyView()( _T_,@5 )

recorded history:
     com.example.fragmentlifecycle        I  cb 101963234 onCreate
     com.example.fragmentlifecycle        I   cb 213444936 onAttach 101963234
     com.example.fragmentlifecycle        I  ci 101963234 setContentView 2131427356
     com.example.fragmentli

benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_fragment_lifecycle,1/1,6.0,7.0,14.0,4,yes,3,5,9


import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}

    
frag_destroy_view: OAbsMsg = OAbsMsg(
  CBEnter,
  SubClassMatcher(
    Set(
      "android.app.Fragment",
      "androidx.fragment.app.Fragment",
      "android.support.v4.app.Fragment"
    ),
    "void onDestroyView\\(\\)",
    "onDestroyView"
  ),
  List(_T_, NamedPureVar("a")),
  false
)
frag_create_view: OAbsMsg = OAbsMsg(
  CBEnter,
  SubClassMatcher(
    Set(
      "android.app.Fragment",
      "androidx.fragment.app.Fragment",
      "android.support.v4.app.Fragment"
    ),
    "android.view.View onCreateView\\(.*\\)",
    "onCreateView"
  ),
  List(_T_, NamedPureVar("a"), NamedPureVar("d")),
  false
)
inflate: OAbsMsg = OAbsMsg(
  CIExit,
  SubClassMatcher(
    Set("android.view.LayoutInflater"),
    "android.view.View inflate\\(.*\\)",
    "inflate"
  ),
  List(NamedPureVar("v")),
  false
)
frag_des_last: LSSpec = LSSpec(
  List(NamedPureVar("a")),
  List(NamedPureVar("d")),
  OAbsMsg(
    CBEnte

## Null Pointer

In [13]:
val nullptrExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_nullptr"
val nullptrRuntimeStats = runtimeStats(File(nullptrExpBase))
Table(Seq(nullptrRuntimeStats))

found 1 runtime traces


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot
EXN_nullptr,1/1,2.0,7.0,13.0


nullptrExpBase: String = "/home/testApps//ChatGPT_Benchmarks/EXN_nullptr"
nullptrRuntimeStats: RuntimeRow = RuntimeRow(
  /home/testApps/ChatGPT_Benchmarks/EXN_nullptr,
  List(
    CategoryRow(
      "Runtime Inst",
      "ri",
      List(
        ("Witness Ratio", "1/1"),
        ("cb(n)", 2.0F),
        ("ci(n)", 7.0F),
        ("tot", 13.0F)
      )
    )
  )
)

In [29]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState._
import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal
import edu.colorado.plv.bounder.symbolicexecutor.state._

val get_intent = AbsMsg(CIExit, SubClassMatcher(Set("android.app.Activity"), ".* getIntent\\(.*", "getIntent"),v::Nil)
val start_act = AbsMsg(CIExit,SubClassMatcher(Set("android.app.Activity"), ".* startActivity\\(.*", "startActivity"),TopVal::TopVal::v::Nil)
val get_string_extra = AbsMsg(CIExit, SubClassMatcher(Set("android.content.Intent"), ".* getStringExtra\\(.*", "getStringExtra"),a::v::d::Nil)
val put_extra = AbsMsg(CIExit, SubClassMatcher(Set("android.content.Intent"), ".* putExtra\\(.*", "putExtra"),TopVal::v::d::a::Nil)
val get_after_start = LSSpec(v::Nil, Nil, start_act,get_intent)

val internal = And(Or(put_extra, LSConstraint(a, Equals, NullVal)),
Or(Not(put_extra),LSConstraint(a, NotEquals, NullVal)))
val get_put = LSSpec(v::d::a::Nil, Nil,internal, get_string_extra)
object NullPtr{
    val disallow = LSSpec(a::Nil, d::Nil, SDialog.showI,SpecSignatures.Activity_onDestroy_exit)
    val enableSpecs = Set[LSSpec](get_after_start,get_put)
    val outputDir = File("/home/notebooks/reachExpGPT/NullPtr")
    // println(outputDir.exists())
   
    val allMsg = Set[LifeState.OAbsMsg](SpecSignatures.Activity_onCreate_entry, 
                                        SpecSignatures.Activity_onDestroy_entry, 
                                        find_view,
                                        set_content_view,
                                        get_intent,
                                        start_act,
                                        get_string_extra,
                                        put_extra
                                       )
}

def runNullPtr(){
    val outputDir = NullPtr.outputDir
    val disallow = NullPtr.disallow
    val enableSpecs = NullPtr.enableSpecs

    
    val query = ReceiverNonNull(LifeState.Signature("com.example.nullptr.SecondActivity","void onCreate(android.os.Bundle)"),25,Some(".*toUpperCase.*"));

    val inputApk = s"${nullptrExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/NullPtr")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(NullPtr.enableSpecs, 
                                             Set(disallow), 
                                            NullPtr.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runNullPtr()
}

object NullPtrStats{
    println("specs:")
    (NullPtr.enableSpecs + NullPtr.disallow).foreach{printSpec}
    val wistoria = wistoriaStats(File(nullptrExpBase),NullPtr.outputDir, NullPtr.enableSpecs + NullPtr.disallow, Some("logcat.txt"))
}
val nullptrRow = Table(Seq(nullptrRuntimeStats.addCategories(NullPtrStats.wistoria)))

specs:
Ɐp-v . ∃ . O(CIExit I_CIExit_getIntent ( p-v ) -[]-> O(CIExit I_CIExit_startActivity ( _T_,_T_,p-v ) 
Ɐp-v,p-d,p-a . ∃ . O(CIExit I_CIExit_getStringExtra ( p-a,p-v,p-d ) -[]-> [ [O(CIExit I_CIExit_putExtra ( _T_,p-v,p-d,p-a ) || [ p-a == NULL ]] && [(Not O(CIExit I_CIExit_putExtra ( _T_,p-v,p-d,p-a )) || [ p-a != NULL ]] ] 
Ɐp-a . ∃p-d . O(CBExit I_CBExit_ActivityonDestroy ( _T_,p-a ) -[]-> O(CIExit I_CIExit_Dialogshow ( p-d,_T_,p-a ) 

wistoria output:
CBEnter void onCreate(android.os.Bundle)( _T_,@6 )
CIExit void setContentView(( _T_,@6 )
TNew(@4,{3})
TNew(@3,{4})
TNew(@7,{4})
CIExit  putExtra(( _T_,@4,@3,@7 )
CIExit  startActivity(( _T_,_T_,@4 )
CBEnter void onCreate(android.os.Bundle)( _T_,@8 )
CIExit void setContentView(( _T_,@8 )
CIExit  getIntent(( @4 )
TNew(@2,{4})
CIExit  getStringExtra(( NULL,@4,@2 )
CIExit  findViewById(( @10,@8 )

recorded history:
     com.example.nullptr                  I  cb 2538402 onCreate
     com.example.nullptr                  I  ci 2538402

benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_nullptr,1/1,2.0,7.0,13.0,6,yes,2,7,13


import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}

import edu.colorado.plv.bounder.lifestate.LifeState._

import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal

import edu.colorado.plv.bounder.symbolicexecutor.state._


get_intent: OAbsMsg = OAbsMsg(
  CIExit,
  SubClassMatcher(Set("android.app.Activity"), ".* getIntent\\(.*", "getIntent"),
  List(NamedPureVar("v")),
  false
)
start_act: OAbsMsg = OAbsMsg(
  CIExit,
  SubClassMatcher(
    Set("android.app.Activity"),
    ".* startActivity\\(.*",
    "startActivity"
  ),
  List(_T_, _T_, NamedPureVar("v")),
  false
)
get_string_extra: OAbsMsg = OAbsMsg(
  CIExit,
  SubClassMatcher(
    Set("android.content.Intent"),
    ".* getStringExtra\\(.*",
    "getStringExtra"
  ),
  List(NamedPureVar("a"), NamedPureVar("v"), NamedPureVar("d")),
  false
)
put_extra: OAbsMsg = OAbsMsg(
  CIExit,
  SubClassMatcher(Set("android.content.Intent"), ".* putExtra\\(.*", "putExtra"),
  List(_T_, NamedPureVar("v"), NamedPu

## Inefficient Network Call

In [15]:
val ineffNetBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_inefficient_network"
val ineffNetRuntimeStats = runtimeStats(File(ineffNetBase))
Table(Seq(ineffNetRuntimeStats))

found 1 runtime traces


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot
EXN_inefficient_network,1/1,5.0,18.0,32.0


ineffNetBase: String = "/home/testApps//ChatGPT_Benchmarks/EXN_inefficient_network"
ineffNetRuntimeStats: RuntimeRow = RuntimeRow(
  /home/testApps/ChatGPT_Benchmarks/EXN_inefficient_network,
  List(
    CategoryRow(
      "Runtime Inst",
      "ri",
      List(
        ("Witness Ratio", "1/1"),
        ("cb(n)", 5.0F),
        ("ci(n)", 18.0F),
        ("tot", 32.0F)
      )
    )
  )
)

In [30]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState._
import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal
import edu.colorado.plv.bounder.symbolicexecutor.state._

object IneffNet{
    val fetchSig = SubClassMatcher(Set("com.android.volley.toolbox.Volley"), ".*newRequestQueue.*", "fetch_articles")
    val fetchI = AbsMsg(CIExit, fetchSig, TopVal::TopVal::t::Nil)
    val fetchIE = AbsMsg(CIEnter,fetchSig, TopVal::TopVal::t::Nil)
    val disallow = LSSpec(t::Nil, Nil, fetchI, fetchIE )
    val resume_after_create = LSSpec(a::Nil,Nil,SpecSignatures.Activity_onCreate_entry,SpecSignatures.Activity_onResume_entry)
    val enableSpecs = Set[LSSpec](resume_after_create)
    val outputDir = File("/home/notebooks/reachExpGPT/IneffNet")
    // println(outputDir.exists())

    val allMsg = Set[LifeState.OAbsMsg](
                                       SpecSignatures.Activity_onCreate_entry,
                                       fetchI,
                                       fetchIE,
        find_view,
        SpecSignatures.Activity_onResume_entry
    )
}

def runIneffNet(){
    val outputDir = IneffNet.outputDir
    val disallow = IneffNet.disallow
    val enableSpecs = IneffNet.enableSpecs

    val query = //Reachable(LifeState.Signature("com.example.inefficientnetwork.MainActivity", "void fetchArticles()"),67)
    DisallowedCallin("com.example.inefficientnetwork.MainActivity", "void fetchArticles()", IneffNet.disallow)
    //Reachable(LifeState.Signature("com.example.inefficientnetwork.MainActivity", "void fetchArticles()"),104)
    val inputApk = s"${ineffNetBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/IneffNet")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(IneffNet.enableSpecs, 
                                             Set(disallow), 
                                            IneffNet.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runIneffNet()
}

object IneffNetStats{
    println("specs:")
    (IneffNet.enableSpecs + IneffNet.disallow).foreach{printSpec}
    val wistoria = wistoriaStats(File(ineffNetBase),IneffNet.outputDir, IneffNet.enableSpecs + IneffNet.disallow, Some("logcat.txt"))
}
val ineffNetRow = Table(Seq(ineffNetRuntimeStats.addCategories(IneffNetStats.wistoria)))

specs:
Ɐp-a . ∃ . O(CBEnter I_CBEnter_ActivityonResume ( _T_,p-a ) -[]-> O(CBEnter I_CBEnter_ActivityonCreate ( _T_,p-a ) 
Ɐp-t . ∃ . O(CIEnter I_CIEnter_fetcharticles ( _T_,_T_,p-t ) -[]-> O(CIExit I_CIExit_fetcharticles ( _T_,_T_,p-t ) 

wistoria output:
CBEnter void onCreate(android.os.Bundle)( _T_,@4 )
CIExit  findViewById(( @7,@4 )
CIExit  findViewById(( @0,@4 )
CIExit newRequestQueue(( _T_,_T_,@4 )
CBEnter void onCreate(android.os.Bundle)( _T_,@4 )
CIExit  findViewById(( @8,@4 )
CIExit  findViewById(( @6,@4 )
CIEnter newRequestQueue(( _T_,_T_,@4 )

recorded history:
     com.example.inefficientnetwork       I  cb 36503278 onCreate
     com.example.inefficientnetwork       I  83857799 = ci 36503278 findViewById 2131165362
     com.example.inefficientnetwork       I   76695732 = new String 61340893
     com.example.inefficientnetwork       I  263831331 = ci 40188960 newRequestQueue 36503278
     com.example.inefficientnetwork       I   30211998 = new StringRequest 207563391 7669573

benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_inefficient_network,1/1,5.0,18.0,32.0,4,yes,2,5,7


import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}

import edu.colorado.plv.bounder.lifestate.LifeState._

import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal

import edu.colorado.plv.bounder.symbolicexecutor.state._


defined object IneffNet
defined function runIneffNet
defined object IneffNetStats

## Long Running Task

In [17]:
val longTaskExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_longRunningTask"
val longTaskRuntimeStats = runtimeStats(File(longTaskExpBase))
Table(Seq(longTaskRuntimeStats))

found 1 runtime traces


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot
EXN_longRunningTask,1/1,2.0,6.0,8.0


longTaskExpBase: String = "/home/testApps//ChatGPT_Benchmarks/EXN_longRunningTask"
longTaskRuntimeStats: RuntimeRow = RuntimeRow(
  /home/testApps/ChatGPT_Benchmarks/EXN_longRunningTask,
  List(
    CategoryRow(
      "Runtime Inst",
      "ri",
      List(
        ("Witness Ratio", "1/1"),
        ("cb(n)", 2.0F),
        ("ci(n)", 6.0F),
        ("tot", 8.0F)
      )
    )
  )
)

In [31]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState._
import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal
import edu.colorado.plv.bounder.symbolicexecutor.state._

object LongTask{
    val observe_on = AbsMsg(CIExit, SubClassMatcher(Set("io.reactivex.Observable"), ".* observeOn\\(.*", "observeOn"),TopVal::Nil)
    val from_callable = AbsMsg(CIExit, SubClassMatcher(Set("io.reactivex.Observable"), ".* fromCallable\\(.*", "fromCallable"),TopVal::Nil)
    val subscribe_on = AbsMsg(CIEnter, SubClassMatcher(Set("io.reactivex.Observable"), ".* subscribeOn\\(.*", "subscribeOn"),TopVal::Nil)
    val disallow = LSSpec(Nil,a::Nil,NS(SpecSignatures.Activity_onCreate_entry,SpecSignatures.Activity_onCreate_exit),subscribe_on )
    val enableSpecs = Set[LSSpec]()
    val outputDir = File("/home/notebooks/reachExpGPT/LongTask")
    // println(outputDir.exists())
   
    val allMsg = Set[LifeState.OAbsMsg](SpecSignatures.Activity_onCreate_entry, 
                                        SpecSignatures.Activity_onDestroy_entry,
                                        SpecSignatures.Activity_onCreate_exit,
                                        observe_on,
                                        subscribe_on,
                                        from_callable
                                       )
}

def runLongTask(){
    val outputDir = LongTask.outputDir
    val disallow = LongTask.disallow
    val enableSpecs = LongTask.enableSpecs

    
    val query = DisallowedCallin("com.example.longtask.MainActivity", "void processDataRxIncorrectly()", disallow)
    //Reachable(LifeState.Signature("com.example.longtask.MainActivity", "void processDataRxIncorrectly()"),61)

    val inputApk = s"${longTaskExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/LongTask")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(LongTask.enableSpecs, 
                                             Set(disallow), 
                                            LongTask.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runLongTask()
}

object LongTaskStats{
    println("specs:")
    (LongTask.enableSpecs + LongTask.disallow).foreach{printSpec}
    val wistoria = wistoriaStats(File(longTaskExpBase),LongTask.outputDir, LongTask.enableSpecs + LongTask.disallow, Some("logcat.txt"))
}
val longTaskRow = Table(Seq(longTaskRuntimeStats.addCategories(LongTaskStats.wistoria)))

specs:
Ɐ . ∃p-a . O(CIEnter I_CIEnter_subscribeOn ( _T_ ) -[]-> NS( O(CBEnter I_CBEnter_ActivityonCreate ( _T_,p-a ) , O(CBExit I_CBExit_ActivityonCreate ( _T_,p-a ) ) 

wistoria output:
CBEnter void onCreate(android.os.Bundle)( _T_,@1 )
CIExit  fromCallable(( _T_ )
CIExit  observeOn(( _T_ )
CIEnter  subscribeOn(( _T_ )

recorded history:
     com.example.longtask       I  cb 45378472 onCreate
     com.example.longtask       I ci 45378472 setContentView 2131361820
     com.example.longtask       I 12341539 = ci 45378472 findViewById 2131165460
     com.example.longtask       I 83453472 = ci 135162841 fromCallable 97925534
     com.example.longtask       I 267967615 = ci 83453472 observeOn 157205580
     com.example.longtask       I 200338581 = ci 267967615 subscribeOn 157205580
     com.example.longtask       I 233329066 = ci 200338581 subscribe 138985883 182049080
     com.example.longtask       I cb 97925534 call


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_longRunningTask,1/1,2.0,6.0,8.0,3,yes,1,2,3


import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}

import edu.colorado.plv.bounder.lifestate.LifeState._

import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal

import edu.colorado.plv.bounder.symbolicexecutor.state._


defined object LongTask
defined function runLongTask
defined object LongTaskStats

## Configuration Changes

In [19]:
val cfgChangeExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_configuration_changes"
val cfgChangeRuntimeStats = runtimeStats(File(cfgChangeExpBase))
Table(Seq(cfgChangeRuntimeStats))

found 1 runtime traces


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot
EXN_configuration_changes,1/1,2.0,3.0,5.0


cfgChangeExpBase: String = "/home/testApps//ChatGPT_Benchmarks/EXN_configuration_changes"
cfgChangeRuntimeStats: RuntimeRow = RuntimeRow(
  /home/testApps/ChatGPT_Benchmarks/EXN_configuration_changes,
  List(
    CategoryRow(
      "Runtime Inst",
      "ri",
      List(
        ("Witness Ratio", "1/1"),
        ("cb(n)", 2.0F),
        ("ci(n)", 3.0F),
        ("tot", 5.0F)
      )
    )
  )
)

In [32]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState._
import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal
import edu.colorado.plv.bounder.symbolicexecutor.state._

object CfgChange{
    val create_destroy = LSSpec(a::Nil,Nil, SpecSignatures.Activity_onCreate_exit,SpecSignatures.Activity_onDestroy_entry)
    val put_string = AbsMsg(CIExit, SubClassMatcher(Set("android.os.Bundle"), ".*putString.*","putString"),TopVal::Nil)
    val enableSpecs = Set[LSSpec](create_destroy)
    val disallow = LSSpec(a::Nil,Nil,NS(SpecSignatures.Activity_onCreate_exit, put_string), SpecSignatures.Activity_onDestroy_exit)
    val outputDir = File("/home/notebooks/reachExpGPT/ConfigChange")
    // println(outputDir.exists())
   
    val allMsg = Set[LifeState.OAbsMsg](SpecSignatures.Activity_onCreate_entry, 
                                        SpecSignatures.Activity_onDestroy_entry,
                                        SpecSignatures.Activity_onCreate_exit,
                                        put_string
                                       )
}


def runCfgChange(){
    val outputDir = CfgChange.outputDir
    val disallow = CfgChange.disallow
    val enableSpecs = CfgChange.enableSpecs

    val query = DisallowedCallin("com.example.configchange.MainActivity", "void onDestroy()", disallow)
    //Reachable(LifeState.Signature("com.example.configchange.MainActivity", "void onDestroy()"),53)

    val inputApk = s"${cfgChangeExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/ConfigChange")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(CfgChange.enableSpecs, 
                                             Set(disallow), 
                                            CfgChange.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runCfgChange()
}

object CfgChangeStats{
    println("specs:")
    (CfgChange.enableSpecs + CfgChange.disallow).foreach{printSpec}
    val wistoria = wistoriaStats(File(cfgChangeExpBase),CfgChange.outputDir, CfgChange.enableSpecs + CfgChange.disallow, Some("logcat.txt"))
}
val cfgChangeRow = Table(Seq(cfgChangeRuntimeStats.addCategories(CfgChangeStats.wistoria)))


specs:
Ɐp-a . ∃ . O(CBEnter I_CBEnter_ActivityonDestroy ( _T_,p-a ) -[]-> O(CBExit I_CBExit_ActivityonCreate ( _T_,p-a ) 
Ɐp-a . ∃ . O(CBExit I_CBExit_ActivityonDestroy ( _T_,p-a ) -[]-> NS( O(CBExit I_CBExit_ActivityonCreate ( _T_,p-a ) , O(CIExit I_CIExit_putString ( _T_ ) ) 

wistoria output:
CBEnter void onCreate(android.os.Bundle)( _T_,@2 )
CBExit void onCreate(android.os.Bundle)( _T_,@2 )
CBEnter void onDestroy()( _T_,@2 )

recorded history:
     com.example.configchange             I  cb 178266702 onCreate
     com.example.configchange             I  ci 178266702 setContentView 2131361820
     com.example.configchange             I  98305730 = ci 178266702 findViewById 2131165477
     com.example.configchange             I  80358611 = ci 178266702 findViewById 2131165326
     com.example.configchange             I  cb 178266702 onDestroy


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_configuration_changes,1/1,2.0,3.0,5.0,4,yes,2,0,2


import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}

import edu.colorado.plv.bounder.lifestate.LifeState._

import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal

import edu.colorado.plv.bounder.symbolicexecutor.state._


defined object CfgChange
defined function runCfgChange
defined object CfgChangeStats

## Table 3 Producing Realistic Execution Histories

In [33]:
def consolidateTables(tables: Seq[Table]) = {
    Table(tables.map(x => x.v).flatten)
}

val all_tables = Seq(bitmapRow,cfgChangeRow,dialogRow,execRow,fragRow,ineffNetRow,longTaskRow,memLeakRow, nullptrRow)
consolidateTables(all_tables)

// Note that EXN_activity_leak_noStatic is "memory leak" in the paper.


benchmark name,ri:Witness Ratio,ri:cb(n),ri:ci(n),ri:tot,wi:specified msgs(n),wi:realistic,wi:cb(n),wi:ci(n),wi:tot
EXN_bitmap_mishandling,1/1,15.0,32.0,54.0,7,yes,5,9,17
EXN_configuration_changes,1/1,2.0,3.0,5.0,4,yes,2,0,2
EXN_dialog_originally_created_here,1/2,4.0,6.0,14.5,3,yes,2,4,8
EXN_execute_twice,1/2,9.5,15.5,26.0,5,yes,3,2,6
EXN_fragment_lifecycle,1/1,6.0,7.0,14.0,4,yes,3,5,9
EXN_inefficient_network,1/1,5.0,18.0,32.0,4,yes,2,5,7
EXN_longRunningTask,1/1,2.0,6.0,8.0,3,yes,1,2,3
EXN_activity_leak_noStatic,1/2,5.5,2.0,11.5,3,yes,3,1,6
EXN_nullptr,1/1,2.0,7.0,13.0,6,yes,2,7,13


defined function consolidateTables
all_tables: Seq[Table] = List(
  Table(
    List(
      RuntimeRow(
        /home/testApps/ChatGPT_Benchmarks/EXN_bitmap_mishandling,
        List(
          CategoryRow(
            "Runtime Inst",
            "ri",
            List(
              ("Witness Ratio", "1/1"),
              ("cb(n)", 15.0F),
              ("ci(n)", 32.0F),
              ("tot", 54.0F)
            )
          ),
          CategoryRow(
            "Wistoria",
            "wi",
            List(
              ("specified msgs(n)", 7),
              ("realistic", "yes"),
              ("cb(n)", 5),
              ("ci(n)", 9),
              ("tot", 17)
            )
          )
        )
      )
    )
  ),
  Table(
    List(
      RuntimeRow(
        /home/testApps/ChatGPT_Benchmarks/EXN_configuration_changes,
        List(
          CategoryRow(
            "Runtime Inst",
            "ri",
...

In [43]:
// App info automatic.  Currently unused

if(false){
def apk_info(expName:String) = {
    val cmd = s"java -jar ${jarpath} -m info -c /home/notebooks/reachExpGPT/${expName}/cfg.json"
    BounderUtil.runCmdFileOut(cmd, File(s"/home/notebooks/reachExpGPT/${expName}/"))
    File(s"/home/notebooks/reachExpGPT/${expName}/messages_all.json").contentAsString
}
List("MemLeak",
    "Bitmap",
"DialogCreated",
"IneffNet",
    ).foreach{ benchmark => 
    println(s"${benchmark}: ${apk_info(benchmark)}")
}
}